In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np


# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(28 * 28, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input
        return self.fc(x)


# Generate adversarial examples
def generate_adversarial(model, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, target)
    loss.backward()
    adversarial_data = data + epsilon * data.grad.sign()
    return torch.clamp(adversarial_data, 0, 1)


# Tangent propagation loss
def tangent_prop_loss(model, data, tangent_vectors, target, weight):
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, target)
    tangent_loss = 0.0

    for t in tangent_vectors:
        # Expand tangent vector to match the batch size
        tangent_noise = t.expand_as(data)
        tangent_loss += ((model(data + tangent_noise) - output) ** 2).mean()

    return loss + weight * tangent_loss


# Tangent distance calculation
def tangent_distance(x1, x2, tangents):
    return min(np.linalg.norm((x1 - x2 - t.numpy())) for t in tangents)


# Tangent classifier
def tangent_classifier(x, training_data, labels, tangents):
    distances = [tangent_distance(x, train_x, tangents) for train_x in training_data]
    return labels[np.argmin(distances)]


# Training loop
def train(model, train_loader, optimizer, epsilon, tangents, weight):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Generate adversarial examples
        adv_data = generate_adversarial(model, data, target, epsilon)

        # Reset gradients
        optimizer.zero_grad()

        # Compute losses
        loss = tangent_prop_loss(model, data, tangents, target, weight) + \
               nn.CrossEntropyLoss()(model(adv_data), target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print training progress
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}: Loss = {loss.item()}")


# Main script
if __name__ == "__main__":
    # Data transformation
    transform = transforms.ToTensor()

    # Load MNIST dataset
    train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Model and optimizer
    model = SimpleNN().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # Generate tangent vectors matching the shape of input data
    tangents = [torch.randn(1, 1, 28, 28, device=device) * 0.1 for _ in range(5)]

    # Train the model
    train(model, train_loader, optimizer, epsilon=0.2, tangents=tangents, weight=0.1)


Batch 0: Loss = 6.1383771896362305
Batch 100: Loss = 3.5806150436401367
Batch 200: Loss = 3.0211429595947266
Batch 300: Loss = 2.921473503112793
Batch 400: Loss = 2.755136728286743
Batch 500: Loss = 2.5830039978027344
Batch 600: Loss = 2.663576602935791
Batch 700: Loss = 2.1128323078155518
Batch 800: Loss = 2.9511678218841553
Batch 900: Loss = 2.8296451568603516
